In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# LDDMM landmark stochastic dynamics

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.manifolds.landmarks import *
M = landmarks(6)
print(M)
from src.plotting import *

In [ ]:
# Riemannian structure
from src.Riemannian import metric
metric.initialize(M)

In [ ]:
# example configuration
M.k_sigma = jnp.diag(jnp.array([.5,.5]))

q = M.coords(jnp.vstack((np.linspace(-.5,.5,M.N),np.zeros(M.N))).T.flatten())
v = jnp.array(jnp.vstack((np.zeros(M.N),np.ones(M.N))).T.flatten())
p = M.flat(q,v)
print("q = ", q)
print("p = ", p)

## Brownian Motion

In [ ]:
# coordinate form
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

(ts,xs,charts) = M.Brownian_coords(q,dWs(M.dim,n_steps=1000))

# plot
M.newfig()
M.plot()
M.plot_path(zip(xs,charts))
plt.show()

# plot multiple sample paths
N = 5
xss = np.zeros((N,xs.shape[0],M.dim))
chartss = np.zeros((N,xs.shape[0],q[1].shape[0]))
for i in range(N):
    (ts,xs,charts) = M.Brownian_coords(q,dWs(M.dim,n_steps=xs.shape[0]))
    xss[i] = xs
    chartss[i] = charts

# plot
M.newfig()
M.plot()
colormap = plt.get_cmap('winter')
colors=[colormap(k) for k in np.linspace(0, 1, N)]
for i in range(N):
    M.plot_path(zip(xss[i],chartss[i]),color=colors[i])
M.plotx(q,color='r')
plt.show()

# Langevin equations
see https://arxiv.org/abs/1605.09276

In [ ]:
from src.stochastics import Langevin
Langevin.initialize(M)

(ts,qps,charts) = M.Langevin(q,p,.5,.25,dWs(M.dim))

# plot
M.newfig()
M.plot()
M.plot_path(zip(qps[:,0,:],charts))
plt.show()

## Stochastic EPDiff / Eulerian

In [ ]:
# define noise field grid
minx = -1; maxx = 1
miny = -1; maxy = 1
pts = 5

X, Y = jnp.meshgrid(np.linspace(minx,maxx,pts),np.linspace(miny,maxy,pts))
xy = jnp.vstack([X.ravel(), Y.ravel()]).T
sigmas_x = jnp.hstack((xy,xy)).reshape((-1,2))
sigmas_a = .05*jnp.tile(np.eye(2),(sigmas_x.shape[0]//2,1))
#sigmas_x = np.array([[0.,0.]])
#sigmas_a = np.array([[.1,0.]])
J = sigmas_x.shape[0]

# noise kernels
k_alpha = 1.
k_sigma = jnp.diag(jnp.array([(maxx-minx)/(pts-1),(maxy-miny)/(pts-1)]))
inv_k_sigma = jnp.linalg.inv(k_sigma)
k = lambda x: M.k_alpha*jnp.exp(-.5*jnp.square(jnp.tensordot(x,M.inv_k_sigma,(x.ndim-1,1))).sum(x.ndim-1))

# initialize with specified noise kernel
from src.stochastics import Eulerian
Eulerian.initialize(M,k=k)

# integrate
(ts,qps,charts) = M.Eulerian(q,p,sigmas_x,sigmas_a,dWs(J))

# plot
M.newfig()
M.plot()
M.plot_path(zip(qps[:,0,:],charts))
plt.plot(sigmas_x[:,0],sigmas_x[:,1],'x')
plt.show()